This post follows the [Learning Git Branching](https://learngitbranching.js.org) game, in which we use git commands to reach the desired state for the git repository at each level.

::: {.content-hidden}
- https://missing.csail.mit.edu/2020/version-control/
- https://xosh.org/explain-git-in-simple-words/
:::

Let's get to the same stage as the beginning of the game.

- First, we need to create a local git repository:

In [ ]:
!mkdir example
%cd example
!ls -la; git init

/Users/afonsomm/blog/posts/git/example
total 0
drwxr-xr-x@ 2 afonsomm  staff   64 Apr 16 11:44 .
drwxr-xr-x  8 afonsomm  staff  256 Apr 16 11:44 ..
Initialized empty Git repository in /Users/afonsomm/blog/posts/git/example/.git/


- We created a directory "example" with an empty git repository (using the `git init` command).

>- *Sidenote*: In notebooks, the syntax `!command` opens a new shell in a different process, executes the command in that shell, and terminates the process running it right after, while the `%command` syntax is used for *magic commands*, which are specified by the IPython kernel, including some shell commands such as `cd`, and also timing commands, plotting, etc. Suppose we want to change to a given directory in a lasting way. In that case, we need to run the `%command`, since with the `!command`, the current process will be unchanged, and the directory is changed on the newly created one, which also terminates right after command execution is over.

- Now, let's add our first change to the repository:

In [ ]:
!echo "hello world" >> file.txt; git add file.txt; git commit -m "C0"

[main (root-commit) 4abe474] C0
 1 file changed, 1 insertion(+)
 create mode 100644 file.txt


- Here, we created a text file "file.txt" containing "hello world". Then we **staged** that file using the `git add` command, which means "add the mentioned changed files/directories (in this case "file.txt") in their current state to the list of changes to be commited". Finally, we used the `git commit` command to take a **snapshot** of all the tracked files in our working directory (i.e., the one where ".git" is located, in this case, "./example").

In [ ]:
!git log --all --graph --decorate

* commit 4abe4740bdd9598bd5c7cc79a987dd5f0554c770 (HEAD -> main)
  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
  Date:   Tue Apr 16 11:44:37 2024 +0100
  
      C0


- The `git log` command allows us to see the entire history of commits for the current **branch** we are in (in this case, "main"). A branch is a pointer to a given commit. If we add the `--all` flag, we print out the commit history for every branch. With the `--graph` flag, we visualize the commits with dashed lines indicating what is/are the previous commit(s) a given commit depends on, meaning what are its ancestor commits. With the `--decorate` flag, we give some colors to the output and print out to which *commit* each branch is pointing and which *commit* we are currently **checking out**, defined by the *HEAD* pointer. However, for new versions of git, using this flag does not make a difference in the output.

>- *Sidenote*: when the *HEAD* pointer is not pointing to the same commit where a *branch* is pointing to, then it is called **detached (HEAD)**.

In [ ]:
!echo "hello world changed" >> file.txt; git add file.txt; git commit -m "C1"

[main d6f8c4c] C1
 1 file changed, 1 insertion(+)


In [ ]:
!git log --all --graph --decorate

* commit d6f8c4ce16a5db9489f3633584a1e149f7c0ddbd (HEAD -> main)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:38 2024 +0100
| 
|     C1
| 
* commit 4abe4740bdd9598bd5c7cc79a987dd5f0554c770
  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
  Date:   Tue Apr 16 11:44:37 2024 +0100
  
      C0


### Level 1

- To accomplish this level, we need to make a change in the working directory, stage that change, and commit it 2 times:

In [ ]:
!echo "hello world changed 1" >> file.txt; git add file.txt; git commit -m "C2"
!echo "hello world changed 2" >> file.txt; git add file.txt; git commit -m "C3"

[main eadba0c] C2
 1 file changed, 1 insertion(+)
[main aa84653] C3
 1 file changed, 1 insertion(+)


In [ ]:
!git log --all --graph --decorate

* commit aa84653c82524cf76c3b0e14604469c4e89c7a2f (HEAD -> main)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:40 2024 +0100
| 
|     C3
| 
* commit eadba0c6cff8bebc16b48ae50b09a574eb0c0e37
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:40 2024 +0100
| 
|     C2
| 
* commit d6f8c4ce16a5db9489f3633584a1e149f7c0ddbd
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:38 2024 +0100
| 
|     C1
| 
* commit 4abe4740bdd9598bd5c7cc79a987dd5f0554c770
  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
  Date:   Tue Apr 16 11:44:37 2024 +0100
  
      C0


In [ ]:
!cat file.txt

hello world
hello world changed
hello world changed 1
hello world changed 2


### Level 2

- Now we need to get back to the 2 commit state of our repository, as we were at the beginning of level 1. To do that, we have to roll back to the "C1" commit and throw away the subsequent ones ("C2" and "C3"). We can do that using the `git reset <commit-id>` command and we can execute in three main ways:
    - Using the `--soft` flag which moves the branch where HEAD is pointing to the specified commit, but keeps the Index and Working Directory intact, meaning the changes made after the specified commit are staged but not commited.
    - Using the `--mixed` flag (default), which also updates the **Index** (the proposed next commit) with the contents of that commit, but does not update the Working Directory, so we need to specify which changes we want to stage.
    - Using the `--hard` flag, which also updates the Working Directory, meaning the changes made in subsequent commits after the one specified are thrown away, for the current branch (where HEAD is pointing to). In this case, if we reset to commit "C1" while HEAD is pointing to "main", since no other branches are pointing to subsequent commits after "C1", these become untracked and so are completely thrown away, which is what we want. 

>- *Sidenote*: The Index is a file (".git/index") that has the file contents that were checked out into the working directory (1) and what they looked like when they were initially checked out (2). When we change and stage a file using the `git add` command, we replace its contents in (1) with its new version. The following `git commit` command will use this Index to create a new snapshot of the working directory.
>- *Sidenote*: To reset to the HEAD's parent commit, we use `git reset HEAD~`.

In [ ]:
!git reset --hard b20af6721a6a78e3e3077c3c021b345f39f00fac
!git log --all --graph
!cat file.txt

fatal: Could not parse object 'b20af6721a6a78e3e3077c3c021b345f39f00fac'.
* commit aa84653c82524cf76c3b0e14604469c4e89c7a2f (HEAD -> main)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:40 2024 +0100
| 
|     C3
| 
* commit eadba0c6cff8bebc16b48ae50b09a574eb0c0e37
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:40 2024 +0100
| 
|     C2
| 
* commit d6f8c4ce16a5db9489f3633584a1e149f7c0ddbd
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:38 2024 +0100
| 
|     C1
| 
* commit 4abe4740bdd9598bd5c7cc79a987dd5f0554c770
  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
  Date:   Tue Apr 16 11:44:37 2024 +0100
  
      C0
hello world
hello world changed
hello world changed 1
hello world changed 2


- To complete this level, we create a new *branch* (using `git branch`) and then *check it out* (using `git checkout`), meaning we tell git to point *HEAD* to the *main branch*. This will populate the *Index* with the *commit* content where *main* is pointing and then update the working directory.

>- Sidenote: we can create a *branch* and *check it out* using the command `git checkout -b <name-of-branch>`. 

In [ ]:
!git branch bugFix; git checkout bugFix
!git log --all --graph

Switched to branch 'bugFix'
* commit aa84653c82524cf76c3b0e14604469c4e89c7a2f (HEAD -> bugFix, main)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:40 2024 +0100
| 
|     C3
| 
* commit eadba0c6cff8bebc16b48ae50b09a574eb0c0e37
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:40 2024 +0100
| 
|     C2
| 
* commit d6f8c4ce16a5db9489f3633584a1e149f7c0ddbd
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:38 2024 +0100
| 
|     C1
| 
* commit 4abe4740bdd9598bd5c7cc79a987dd5f0554c770
  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
  Date:   Tue Apr 16 11:44:37 2024 +0100
  
      C0


### Level 3

- Branches allow us to quickly switch between project versions and work on different things in isolation. This removes the possibility of conflicts between the changes made in each thing arising while working on them. We then need to merge these changes together into the main version of the project.

- At this level, we use the `git merge` command, which creates a commit with two parents, including both project versions. When we merge two branches, we may need to resolve conflicts between their work.

- To get to the same stage as the beginning of this level, we would undo and redo what we did in the previous level, so let's just keep going.
- To complete this level, we need to commit from the "bugFix" branch, then check out and commit from the "main" branch, and finally merge the commits where each branch is pointing to while in the "main" branch, so this one points to the created merged commit:

In [ ]:
!echo "hello world changed 3" >> file.txt; git add file.txt; git commit -m "C2"
!git log --all --graph

[bugFix 672d6b3] C2
 1 file changed, 1 insertion(+)
* commit 672d6b3bfe65331fb1944eaffcfaafb349e33bf2 (HEAD -> bugFix)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:44 2024 +0100
| 
|     C2
| 
* commit aa84653c82524cf76c3b0e14604469c4e89c7a2f (main)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:40 2024 +0100
| 
|     C3
| 
* commit eadba0c6cff8bebc16b48ae50b09a574eb0c0e37
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:40 2024 +0100
| 
|     C2
| 
* commit d6f8c4ce16a5db9489f3633584a1e149f7c0ddbd
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:38 2024 +0100
| 
|     C1
| 
* commit 4abe4740bdd9598bd5c7cc79a987dd5f0554c770
  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
  Date:   Tue Apr 16 11:44:37 2024 +0100
  
      C0


In [ ]:
!git checkout main; echo "hello world changed 4" >> file1.txt; git add file1.txt; git commit -m "C3"
!git log --all --graph

Switched to branch 'main'
[main af49e06] C3
 1 file changed, 1 insertion(+)
 create mode 100644 file1.txt
* commit af49e06f42917538d6df2684bda8c654b702f497 (HEAD -> main)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:46 2024 +0100
| 
|     C3
|   
| * commit 672d6b3bfe65331fb1944eaffcfaafb349e33bf2 (bugFix)
|/  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
|   Date:   Tue Apr 16 11:44:44 2024 +0100
|   
|       C2
| 
* commit aa84653c82524cf76c3b0e14604469c4e89c7a2f
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:40 2024 +0100
| 
|     C3
| 
* commit eadba0c6cff8bebc16b48ae50b09a574eb0c0e37
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:40 2024 +0100
| 
|     C2
| 
* commit d6f8c4ce16a5db9489f3633584a1e149f7c0ddbd
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:38 2024 +0100
| 
|     C1
| 
* commit 4abe4740bdd9598bd5c7cc79a987dd5f05

In [ ]:
!git merge bugFix -m "C4"
!git log --all --graph

Merge made by the 'ort' strategy.
 file.txt | 1 +
 1 file changed, 1 insertion(+)
*   commit 4aa4729a84d7b065474892f9bdc82bc3c090f30c (HEAD -> main)
|\  Merge: af49e06 672d6b3
| | Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| | Date:   Tue Apr 16 11:44:47 2024 +0100
| | 
| |     C4
| | 
| * commit 672d6b3bfe65331fb1944eaffcfaafb349e33bf2 (bugFix)
| | Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| | Date:   Tue Apr 16 11:44:44 2024 +0100
| | 
| |     C2
| | 
* | commit af49e06f42917538d6df2684bda8c654b702f497
|/  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
|   Date:   Tue Apr 16 11:44:46 2024 +0100
|   
|       C3
| 
* commit aa84653c82524cf76c3b0e14604469c4e89c7a2f
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:40 2024 +0100
| 
|     C3
| 
* commit eadba0c6cff8bebc16b48ae50b09a574eb0c0e37
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:40 2024 +0100
| 
|     C2
| 
* commit d6f8c4ce1

### Level 4

- Rebasing takes an history of commits and sets their new base commit.

- Let's get to the same stage as the beginning of this level. To do that we need to reset again while on the main branch, and force delete the bugFix branch:

In [ ]:
!git reset --hard d6f8c4ce16a5db9489f3633584a1e149f7c0ddbd; git branch -D bugFix; git log --all --graph

HEAD is now at d6f8c4c C1
Deleted branch bugFix (was 672d6b3).
* commit d6f8c4ce16a5db9489f3633584a1e149f7c0ddbd (HEAD -> main)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:38 2024 +0100
| 
|     C1
| 
* commit 4abe4740bdd9598bd5c7cc79a987dd5f0554c770
  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
  Date:   Tue Apr 16 11:44:37 2024 +0100
  
      C0


- To complete this level, we need to create the bugFix branch again, commit in it, move to the main branch, commit in it, move to the bugFix branch, and then use the `git rebase` command:

In [ ]:
!git checkout -b bugFix; echo "Hello world changed 1" >> file.txt; git add file.txt; git commit -m "C2"
!git checkout main; echo "Hello world" >> file1.txt; git add file1.txt; git commit -m "C3"
!git log --all --graph

Switched to a new branch 'bugFix'
[bugFix af1fb02] C2
 1 file changed, 1 insertion(+)
Switched to branch 'main'
[main 62661e5] C3
 1 file changed, 1 insertion(+)
 create mode 100644 file1.txt
* commit af1fb0205c749e87a938ce4f2d2343bb248ffc57 (bugFix)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:45:00 2024 +0100
| 
|     C2
|   
| * commit 62661e57b270811d8e04f2ab31f1a2c9a45f9414 (HEAD -> main)
|/  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
|   Date:   Tue Apr 16 11:45:00 2024 +0100
|   
|       C3
| 
* commit d6f8c4ce16a5db9489f3633584a1e149f7c0ddbd
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:38 2024 +0100
| 
|     C1
| 
* commit 4abe4740bdd9598bd5c7cc79a987dd5f0554c770
  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
  Date:   Tue Apr 16 11:44:37 2024 +0100
  
      C0


In [ ]:
!git checkout bugFix; git rebase main
!git log --all --graph

Switched to branch 'bugFix'
Successfully rebased and updated refs/heads/bugFix.
* commit 78482701d00947592e903cc20ea233c3f053218c (HEAD -> bugFix)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:45:00 2024 +0100
| 
|     C2
| 
* commit 62661e57b270811d8e04f2ab31f1a2c9a45f9414 (main)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:45:00 2024 +0100
| 
|     C3
| 
* commit d6f8c4ce16a5db9489f3633584a1e149f7c0ddbd
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:38 2024 +0100
| 
|     C1
| 
* commit 4abe4740bdd9598bd5c7cc79a987dd5f0554c770
  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
  Date:   Tue Apr 16 11:44:37 2024 +0100
  
      C0


- This created a clean linear sequence of commits, making the log/history easier to read.

### Level 5

- HEAD corresponds to the currently checked out commit, i.e., the one we are working on top of. Most git commands that change the Working Tree start by changing HEAD.
- A detached HEAD means that it is attached to a commit instead of a branch.
- To get to the same stage as this level we need to reset to C1 on both main and bugFix, commit on main and bugFix separately, and commit again on bugFix:

In [ ]:
!git reset --hard d6f8c4ce16a5db9489f3633584a1e149f7c0ddbd
!git checkout main; git reset --hard d6f8c4ce16a5db9489f3633584a1e149f7c0ddbd
!echo "Hello world changed 1" >> file.txt; git add file.txt; git commit -m "C2"
!git checkout bugFix; echo "Hello world" >> file1.txt; git add file1.txt; git commit -m "C3"
!echo "Hello world changed" >> file1.txt; git add file1.txt; git commit -m "C4"

HEAD is now at d6f8c4c C1
Switched to branch 'main'
HEAD is now at d6f8c4c C1
[main 59e45ed] C2
 1 file changed, 1 insertion(+)
Switched to branch 'bugFix'
[bugFix a782f63] C3
 1 file changed, 1 insertion(+)
 create mode 100644 file1.txt
[bugFix 86c2f7d] C4
 1 file changed, 1 insertion(+)


In [ ]:
!git log --all --graph

* commit 86c2f7df3e7564a65513e7ba0180c7d937453632 (HEAD -> bugFix)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:45:09 2024 +0100
| 
|     C4
| 
* commit a782f6376954bc7fa272ba6227e2bec8389f82cd
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:45:09 2024 +0100
| 
|     C3
|   
| * commit 59e45ed005c84f2342b98f735737ad2536fa0d05 (main)
|/  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
|   Date:   Tue Apr 16 11:45:08 2024 +0100
|   
|       C2
| 
* commit d6f8c4ce16a5db9489f3633584a1e149f7c0ddbd
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:38 2024 +0100
| 
|     C1
| 
* commit 4abe4740bdd9598bd5c7cc79a987dd5f0554c770
  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
  Date:   Tue Apr 16 11:44:37 2024 +0100
  
      C0


- To complete this level we need to checkout the commit where bugFix is pointing to, using its ID:

In [ ]:
!git checkout 86c2f7df3e7564a65513e7ba0180c7d937453632; git log --all --graph

Note: switching to '86c2f7df3e7564a65513e7ba0180c7d937453632'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 86c2f7d C4
* commit 86c2f7df3e7564a65513e7ba0180c7d937453632 (HEAD, bugFix)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:45:09 2024 +0100
| 
|     C4
| 
* commit a782f6376954bc7fa272ba6227e2bec8389f82cd
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:45:09 2024 +0100
| 
|     C3
|   
| * commit 59e45ed005c84f2342b98f735737ad2536fa0d05

- We can see that when HEAD was pointing to bugFix `git log` showed it as `(HEAD -> bugFix)`. Now it shows it as `(HEAD, bugFix)`, so it is not pointing at the branch, but at the commit it self, so it is detached.

### Level 6

- Since commit ids/hashes are relatively big, git only requires us to specify enough characters of that hash until it uniquely identifies that commit. This is still not very convenient, so git provides us relative refs, which allow us to start from somewhere memorable and work from there.

- At this level, we will use `^` which moves upwards one commit.

- Let's get to the same stage as the beginning of the level:

In [ ]:
!git checkout main; git log --all --graph

Previous HEAD position was 86c2f7d C4
Switched to branch 'main'
* commit 86c2f7df3e7564a65513e7ba0180c7d937453632 (bugFix)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:45:09 2024 +0100
| 
|     C4
| 
* commit a782f6376954bc7fa272ba6227e2bec8389f82cd
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:45:09 2024 +0100
| 
|     C3
|   
| * commit 59e45ed005c84f2342b98f735737ad2536fa0d05 (HEAD -> main)
|/  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
|   Date:   Tue Apr 16 11:45:08 2024 +0100
|   
|       C2
| 
* commit d6f8c4ce16a5db9489f3633584a1e149f7c0ddbd
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:38 2024 +0100
| 
|     C1
| 
* commit 4abe4740bdd9598bd5c7cc79a987dd5f0554c770
  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
  Date:   Tue Apr 16 11:44:37 2024 +0100
  
      C0


- To complete this level, we need to checkout the parent of bugFix's commit:

In [ ]:
!git checkout bugFix^

Note: switching to 'bugFix^'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at a782f63 C3


In [ ]:
!git log --all --graph

* commit 86c2f7df3e7564a65513e7ba0180c7d937453632 (bugFix)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:45:09 2024 +0100
| 
|     C4
| 
* commit a782f6376954bc7fa272ba6227e2bec8389f82cd (HEAD)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:45:09 2024 +0100
| 
|     C3
|   
| * commit 59e45ed005c84f2342b98f735737ad2536fa0d05 (main)
|/  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
|   Date:   Tue Apr 16 11:45:08 2024 +0100
|   
|       C2
| 
* commit d6f8c4ce16a5db9489f3633584a1e149f7c0ddbd
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:38 2024 +0100
| 
|     C1
| 
* commit 4abe4740bdd9598bd5c7cc79a987dd5f0554c770
  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
  Date:   Tue Apr 16 11:44:37 2024 +0100
  
      C0


### Level 7

- We can use the `^` operator several times, such as `bugFix^^`, to move up several commits. However, we can use the `~` operator which takes a trailing number and, that way, is more convenient.

- One of the main reasons to use relative refs is to move branches. We can directly reassign a branch to a commit using `git branch -f`, which forces a branch to move.

>- *Sidenote*: In a real git environment this is not allowed for the current checked out branch.

- Let's get to the same stage as the beginning of the level:

In [ ]:
!git branch -f bugFix HEAD; git log --all --graph

* commit a782f6376954bc7fa272ba6227e2bec8389f82cd (HEAD, bugFix)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:45:09 2024 +0100
| 
|     C3
|   
| * commit 59e45ed005c84f2342b98f735737ad2536fa0d05 (main)
|/  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
|   Date:   Tue Apr 16 11:45:08 2024 +0100
|   
|       C2
| 
* commit d6f8c4ce16a5db9489f3633584a1e149f7c0ddbd
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:38 2024 +0100
| 
|     C1
| 
* commit 4abe4740bdd9598bd5c7cc79a987dd5f0554c770
  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
  Date:   Tue Apr 16 11:44:37 2024 +0100
  
      C0


In [ ]:
!git checkout main; echo "Hello world changed 2" >> file.txt; git add file.txt; git commit -m "C4"
!git checkout bugFix; echo "Hello world changed 1" >> file1.txt; git add file1.txt; git commit -m "C5"
!echo "Hello world changed 2" >> file1.txt; git add file1.txt; git commit -m "C6"
!git log --all --graph

Previous HEAD position was a782f63 C3
Switched to branch 'main'
[main 0b253a5] C4
 1 file changed, 1 insertion(+)
Switched to branch 'bugFix'
[bugFix d2450be] C5
 1 file changed, 1 insertion(+)
[bugFix 3806c6c] C6
 1 file changed, 1 insertion(+)
* commit 3806c6ce35da4841587ed2272dd1ccd70690b999 (HEAD -> bugFix)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:45:31 2024 +0100
| 
|     C6
| 
* commit d2450be479728636ea01550c5da0c745376df7bb
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:45:31 2024 +0100
| 
|     C5
| 
* commit a782f6376954bc7fa272ba6227e2bec8389f82cd
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:45:09 2024 +0100
| 
|     C3
|   
| * commit 0b253a577f03107ebd3a8ea4ca7cf2f62d815b72 (main)
| | Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| | Date:   Tue Apr 16 11:45:31 2024 +0100
| | 
| |     C4
| | 
| * commit 59e45ed005c84f2342b98f735737ad2536fa0d05
|/  Author: 

In [ ]:
!git checkout main^; git log --all --graph

Note: switching to 'main^'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 59e45ed C2
* commit 3806c6ce35da4841587ed2272dd1ccd70690b999 (bugFix)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:45:31 2024 +0100
| 
|     C6
| 
* commit d2450be479728636ea01550c5da0c745376df7bb
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:45:31 2024 +0100
| 
|     C5
| 
* commit a782f6376954bc7fa272ba6227e2bec8389f82cd
| Author: Afonso Matoso Magalhães <afonsomm9

- We need to copy the identifier of commit C6 before unreferencing it (`3806c6ce35da4841587ed2272dd1ccd70690b999`).

In [ ]:
!git branch -f bugFix bugFix^; git log --all --graph

* commit d2450be479728636ea01550c5da0c745376df7bb (bugFix)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:45:31 2024 +0100
| 
|     C5
| 
* commit a782f6376954bc7fa272ba6227e2bec8389f82cd
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:45:09 2024 +0100
| 
|     C3
|   
| * commit 0b253a577f03107ebd3a8ea4ca7cf2f62d815b72 (main)
| | Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| | Date:   Tue Apr 16 11:45:31 2024 +0100
| | 
| |     C4
| | 
| * commit 59e45ed005c84f2342b98f735737ad2536fa0d05 (HEAD)
|/  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
|   Date:   Tue Apr 16 11:45:08 2024 +0100
|   
|       C2
| 
* commit d6f8c4ce16a5db9489f3633584a1e149f7c0ddbd
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:38 2024 +0100
| 
|     C1
| 
* commit 4abe4740bdd9598bd5c7cc79a987dd5f0554c770
  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
  Date:   Tue Apr 16 11:44:37 2

- To complete this level we need to point main to C6, HEAD to C1, and bugFix to C0, using relative refs as much as possible:

In [ ]:
!git branch -f main 3806; git log --all --graph

* commit 3806c6ce35da4841587ed2272dd1ccd70690b999 (main)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:45:31 2024 +0100
| 
|     C6
| 
* commit d2450be479728636ea01550c5da0c745376df7bb (bugFix)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:45:31 2024 +0100
| 
|     C5
| 
* commit a782f6376954bc7fa272ba6227e2bec8389f82cd
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:45:09 2024 +0100
| 
|     C3
|   
| * commit 59e45ed005c84f2342b98f735737ad2536fa0d05 (HEAD)
|/  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
|   Date:   Tue Apr 16 11:45:08 2024 +0100
|   
|       C2
| 
* commit d6f8c4ce16a5db9489f3633584a1e149f7c0ddbd
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:38 2024 +0100
| 
|     C1
| 
* commit 4abe4740bdd9598bd5c7cc79a987dd5f0554c770
  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
  Date:   Tue Apr 16 11:44:37 2024 +0100
  

In [ ]:
!git checkout bugFix~2; git branch -f bugFix HEAD^; git log --all --graph

any of your branches:

  59e45ed C2

If you want to keep it by creating a new branch, this may be a good time
to do so with:

 git branch <new-branch-name> 59e45ed

HEAD is now at d6f8c4c C1
* commit 3806c6ce35da4841587ed2272dd1ccd70690b999 (main)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:45:31 2024 +0100
| 
|     C6
| 
* commit d2450be479728636ea01550c5da0c745376df7bb
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:45:31 2024 +0100
| 
|     C5
| 
* commit a782f6376954bc7fa272ba6227e2bec8389f82cd
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:45:09 2024 +0100
| 
|     C3
| 
* commit d6f8c4ce16a5db9489f3633584a1e149f7c0ddbd (HEAD)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:38 2024 +0100
| 
|     C1
| 
* commit 4abe4740bdd9598bd5c7cc79a987dd5f0554c770 (bugFix)
  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
  Date:   Tue Apr 16 11:44:

### Level 8

- There are two ways of reversing changes in git, i.e.:
    - `git reset`, which gets back to a specific commit, discarding subsequent ones. Since some of the commit history may vanish, we should not use this for remote changes in repositories that other people use, but in others cases it's okay.
    - `git revert`, which creates a new commit that reverses the changes made by a specific commit. This does not remove history, so it is good to reverse changes made in remote branches.
 
- Let's get to the same stage as the beginning of the level:

In [ ]:
!git branch -f main HEAD; git branch -D bugFix
!git log --all --graph

Deleted branch bugFix (was 4abe474).
* commit d6f8c4ce16a5db9489f3633584a1e149f7c0ddbd (HEAD, main)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:38 2024 +0100
| 
|     C1
| 
* commit 4abe4740bdd9598bd5c7cc79a987dd5f0554c770
  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
  Date:   Tue Apr 16 11:44:37 2024 +0100
  
      C0


In [ ]:
!git checkout -b pushed; echo "change" >> file.txt; git add file.txt; git commit -m "C2"
!git log --all --graph

Switched to a new branch 'pushed'
[pushed c2bb6ba] C2
 1 file changed, 1 insertion(+)
* commit c2bb6ba7680286da2918097b1cc983d13dd2b452 (HEAD -> pushed)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:49:12 2024 +0100
| 
|     C2
| 
* commit d6f8c4ce16a5db9489f3633584a1e149f7c0ddbd (main)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:38 2024 +0100
| 
|     C1
| 
* commit 4abe4740bdd9598bd5c7cc79a987dd5f0554c770
  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
  Date:   Tue Apr 16 11:44:37 2024 +0100
  
      C0


In [ ]:
!git checkout main; git checkout -b local; echo "change" >> file.txt; git add file.txt; git commit -m "C3"
!git log --all --graph

Switched to branch 'main'
Switched to a new branch 'local'
[local 32d6b19] C3
 1 file changed, 1 insertion(+)
* commit 32d6b190ae74249d46ea0ac1d7a738e816af3b98 (HEAD -> local)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:49:13 2024 +0100
| 
|     C3
|   
| * commit c2bb6ba7680286da2918097b1cc983d13dd2b452 (pushed)
|/  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
|   Date:   Tue Apr 16 11:49:12 2024 +0100
|   
|       C2
| 
* commit d6f8c4ce16a5db9489f3633584a1e149f7c0ddbd (main)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:38 2024 +0100
| 
|     C1
| 
* commit 4abe4740bdd9598bd5c7cc79a987dd5f0554c770
  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
  Date:   Tue Apr 16 11:44:37 2024 +0100
  
      C0


- To complete this level, we need to use the `git reset` command to reverse the local changes made in the local branch back to main, and `git revert` to reverse the changes made in the remote branch pushed back to main:

In [ ]:
!git reset --hard main
!git log --all --graph

HEAD is now at d6f8c4c C1
* commit c2bb6ba7680286da2918097b1cc983d13dd2b452 (pushed)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:49:12 2024 +0100
| 
|     C2
| 
* commit d6f8c4ce16a5db9489f3633584a1e149f7c0ddbd (HEAD -> local, main)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:38 2024 +0100
| 
|     C1
| 
* commit 4abe4740bdd9598bd5c7cc79a987dd5f0554c770
  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
  Date:   Tue Apr 16 11:44:37 2024 +0100
  
      C0


In [ ]:
!git checkout pushed; git revert --no-edit pushed
!git log --all --graph

Switched to branch 'pushed'
[pushed 13b0fb7] Revert "C2"
 Date: Tue Apr 16 11:49:16 2024 +0100
 1 file changed, 1 deletion(-)
* commit 13b0fb7260d5ea890f570d430706d0bcfcc20e32 (HEAD -> pushed)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:49:16 2024 +0100
| 
|     Revert "C2"
|     
|     This reverts commit c2bb6ba7680286da2918097b1cc983d13dd2b452.
| 
* commit c2bb6ba7680286da2918097b1cc983d13dd2b452
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:49:12 2024 +0100
| 
|     C2
| 
* commit d6f8c4ce16a5db9489f3633584a1e149f7c0ddbd (main, local)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:44:38 2024 +0100
| 
|     C1
| 
* commit 4abe4740bdd9598bd5c7cc79a987dd5f0554c770
  Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
  Date:   Tue Apr 16 11:44:37 2024 +0100
  
      C0


### Level 9

- The `git cherry-pick` command allows us to copy a sequence of specific commits to the current checked out branch.

- Let's get to the same point as the beginning of the level: **TODO ...**

In [ ]:
#| echo: false
#| output: false
!git checkout main; git branch -D pushed local

In [ ]:
#| echo: false
#| output: false
!git checkout -b bugFix; echo "1" > file2.txt; git add file2.txt; git commit -m "C2"; echo "2" > file2.txt; git add file2.txt; git commit -m "C3"
!git checkout main
!git checkout -b side; echo "1" > file3.txt; git add file3.txt; git commit -m "C4"; echo "2" > file3.txt; git add file3.txt; git commit -m "C5"
!git checkout main
!git checkout -b another; echo "1" > file4.txt; git add file4.txt; git commit -m "C6"; echo "2" > file4.txt; git add file4.txt; git commit -m "C7"
!git checkout main
!git log --all --graph

Switched to a new branch 'bugFix'
[bugFix b09609a] C2
 1 file changed, 1 insertion(+)
 create mode 100644 file2.txt
[bugFix c373010] C3
 1 file changed, 1 insertion(+), 1 deletion(-)
Switched to branch 'main'
Switched to a new branch 'side'
[side 543b2cb] C4
 1 file changed, 1 insertion(+)
 create mode 100644 file3.txt
[side c3f2ab7] C5
 1 file changed, 1 insertion(+), 1 deletion(-)
Switched to branch 'main'
Switched to a new branch 'another'
[another e5d7fdd] C6
 1 file changed, 1 insertion(+)
 create mode 100644 file4.txt
[another 57e9844] C7
 1 file changed, 1 insertion(+), 1 deletion(-)
Switched to branch 'main'
* commit 57e984493a2c827a12f9377df13f5d426d7f93e0 (another)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:57:13 2024 +0100
| 
|     C7
| 
* commit e5d7fdd3cdd8413060af70494b0b65c781c722c9
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:57:13 2024 +0100
| 
|     C6
|   
| * commit c373010b1a5f143b3ff900ecd2e

::: {.content-hidden}
- To complete this level we need to copy commits C3, C4 and C7 in this order to the main branch:
:::

In [ ]:
#| echo: false
#| output: false
!git cherry-pick bugFix side^ another
!git log --all --graph

CONFLICT (modify/delete): file2.txt deleted in HEAD and modified in 61e8a2a (C3).  Version 61e8a2a (C3) of file2.txt left in tree.
error: could not apply 61e8a2a... C3
hint: After resolving the conflicts, mark them with
hint: "git add/rm <pathspec>", then run
hint: "git cherry-pick --continue".
hint: You can instead skip this commit with "git cherry-pick --skip".
hint: To abort and get back to the state before "git cherry-pick",
hint: run "git cherry-pick --abort".
* commit 57e984493a2c827a12f9377df13f5d426d7f93e0 (another)
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:57:13 2024 +0100
| 
|     C7
| 
* commit e5d7fdd3cdd8413060af70494b0b65c781c722c9
| Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| Date:   Tue Apr 16 11:57:13 2024 +0100
| 
|     C6
|   
| * commit c373010b1a5f143b3ff900ecd2eb628351e29355 (bugFix)
| | Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
| | Date:   Tue Apr 16 11:57:12 2024 +0100
| | 
| |     C3
| | 
| * comm

In [ ]:
#| echo: false
#| output: false
!git branch -D another side bugFix
!git cherry-pick --abort
!git clean -d -x -f

Deleted branch another (was 57e9844).
Deleted branch side (was c3f2ab7).
Deleted branch bugFix (was c373010).


In [ ]:
#| echo: false
#| output: false
!git ls-files --stage

100644 e4aade4828b482cab3237c1a433fe5c6f0081a38 0	file.txt


In [ ]:
#| echo: false
#| output: false
!git log

commit d6f8c4ce16a5db9489f3633584a1e149f7c0ddbd (HEAD -> main)
Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
Date:   Tue Apr 16 11:44:38 2024 +0100

    C1

commit 4abe4740bdd9598bd5c7cc79a987dd5f0554c770
Author: Afonso Matoso Magalhães <afonsomm98@gmail.com>
Date:   Tue Apr 16 11:44:37 2024 +0100

    C0
